In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



Using TensorFlow backend.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd



path  = r"/home/aakashkapoor103/anaconda3/Data/" 

names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']


#['TimeStamp','Response','IOType','LUN','ByteOffset','Size']


all_files = glob.glob(os.path.join(path, "src1_1_sample.csv"))
print(all_files)




['/home/aakashkapoor103/anaconda3/Data/src1_1_sample.csv']


In [5]:
df = pd.read_csv(all_files[0],engine='python',skiprows =4,header=None,na_values=['-1'], index_col=False) 
names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']
df.columns = names
print (len(df))

21267682


In [6]:
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)
print(df.dtypes)
print(len(df))

TimeStamp           int64
Host_Name          object
DiskNumber        float64
Operation_Type     object
ByteOffset        float64
IOSize            float64
Response_Time     float64
DiskNum           float64
dtype: object
21267682


In [7]:
df['IOSize_log'] = np.log2(df['IOSize'])
df['IOSize_log_roundoff']= round(df['IOSize_log'])


print(len(Counter(df['IOSize'])))
print(len(Counter(df['IOSize_log'])))
print(len(Counter(df['IOSize_log_roundoff'])))


129
129
9


In [8]:
# count = 0
# list_generated = []
# delta = 0
# list_roundoff = df['IOSize_log_roundoff'].tolist()
# list_actual = df['IOSize_log'].tolist()
# while(count < len(df)):
#     tmp = list_roundoff[count]
#     tmp_2 = list_actual[count]
#     delta = delta +  abs(tmp - tmp_2)
#     count = count + 1
# print(delta/len(df))

In [9]:

df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
df = df.drop(df.index[-1])




In [10]:
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()
operation_id_map = {}
for i,id in enumerate(a): operation_id_map[id] = i 
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


In [11]:
from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000)
bo_list = []

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list
print(len(Counter(df['ByteOffset_Delta_class'])))
    
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
for i,id in enumerate(a): bo_map[id] = i 
df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])
    
label_list = df['ByteOffset_Delta_Class_1001'] 
    
df['ByteOffset_Delta_Class_1001']  = label_list
    

1001


In [12]:
df = df.drop(df.index[0])

In [13]:
df.head(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum,IOSize_log,IOSize_log_roundoff,ByteOffset_Delta,ByteOffset_Delta_class,ByteOffset_Delta_Class_1001
1,128166372013537059,src1,NaN,Write,3.180110e+09,8192.0,1507.0,1.0,13.000000,13.0,-8192.0,1,1
2,128166372021660589,src1,NaN,Write,3.180118e+09,28672.0,2925.0,1.0,14.807355,15.0,25980928.0,999999,2
3,128166372021662159,src1,NaN,Write,3.154137e+09,4096.0,1355.0,1.0,12.000000,12.0,12288.0,3,3
4,128166372021662166,src1,NaN,Write,3.154125e+09,4096.0,1348.0,1.0,12.000000,12.0,-183537664.0,999999,2
5,128166372023534467,src1,NaN,Write,3.337662e+09,65536.0,4034.0,1.0,16.000000,16.0,-65536.0,5,4


In [14]:
df_backup = df

In [15]:
df.dtypes

TimeStamp                        int64
Host_Name                       object
DiskNumber                     float64
Operation_Type                  object
ByteOffset                     float64
IOSize                         float64
Response_Time                  float64
DiskNum                        float64
IOSize_log                     float64
IOSize_log_roundoff            float64
ByteOffset_Delta               float64
ByteOffset_Delta_class           int64
ByteOffset_Delta_Class_1001      int64
dtype: object

In [16]:
a = df['IOSize_log_roundoff'].unique().tolist()
size_map = {}
for i,id in enumerate(a): size_map[id] = i 
df['Size_Class'] = df['IOSize_log_roundoff'].map(lambda x: size_map[x])
    


In [17]:
# Drop unnecessary coloumns

print("Before drop: {}".format(df.columns))
df.drop(df.columns[[0,1,2,3,6,7,8,10,11]], axis=1,inplace=True) 


Before drop: Index(['TimeStamp', 'Host_Name', 'DiskNumber', 'Operation_Type', 'ByteOffset',
       'IOSize', 'Response_Time', 'DiskNum', 'IOSize_log',
       'IOSize_log_roundoff', 'ByteOffset_Delta', 'ByteOffset_Delta_class',
       'ByteOffset_Delta_Class_1001', 'Size_Class'],
      dtype='object')


In [18]:
df.dtypes

ByteOffset                     float64
IOSize                         float64
IOSize_log_roundoff            float64
ByteOffset_Delta_Class_1001      int64
Size_Class                       int64
dtype: object

In [19]:
from collections import Counter
print(len(Counter(df['Size_Class'])))

8


In [20]:
df.head(5)

,ByteOffset,IOSize,IOSize_log_roundoff,ByteOffset_Delta_Class_1001,Size_Class
1,3.180110e+09,8192.0,13.0,1,0
2,3.180118e+09,28672.0,15.0,2,1
3,3.154137e+09,4096.0,12.0,3,2
4,3.154125e+09,4096.0,12.0,2,2
5,3.337662e+09,65536.0,16.0,4,3


In [21]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_Class'].tolist()
size_test = df[training_pt_1+1:]['Size_Class'].tolist()


In [22]:
lba_train= np.array(lba_train).reshape(-1,1)
lba_test= np.array(lba_test).reshape(-1,1)
size_train= np.array(size_train).reshape(-1,1)
size_test= np.array(size_test).reshape(-1,1)

In [23]:
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

lstm_num_timesteps = 32
    
X_train_lba, y_train_lba = create_dataset2(lba_train, lstm_num_timesteps)
X_test_lba, y_test_lba = create_dataset2(lba_test, lstm_num_timesteps)
X_train_size, y_train_size = create_dataset2(size_train, lstm_num_timesteps)
X_test_size, y_test_size = create_dataset2(size_test, lstm_num_timesteps)

In [24]:
X_train_lba.shape, y_train_lba.shape, X_test_lba.shape,  y_test_lba.shape

((15950696, 32), (15950696, 32), (5316855, 32), (5316855, 32))

In [25]:
X_train_size.shape, y_train_size.shape, X_test_size.shape,  y_test_size.shape

((15950696, 32), (15950696, 32), (5316855, 32), (5316855, 32))

In [26]:
lstm_num_features = 1
lstm_predict_sequences = True
lstm_num_predictions = 32


# X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
# X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    

y_train_lba = np.reshape(y_train_lba, (y_train_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_test_lba = np.reshape(y_test_lba, (y_test_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_train_size = np.reshape(y_train_size, (y_train_size.shape[0], lstm_num_predictions, lstm_num_features))
y_test_size = np.reshape(y_test_size, (y_test_size.shape[0], lstm_num_predictions, lstm_num_features))                        


In [27]:
import pickle 

pickle.dump(X_train_lba, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_X_train_lba.csv", 'wb'), protocol=4)
pickle.dump(X_train_size, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_X_train_size.csv", 'wb'), protocol=4)
pickle.dump(X_test_lba, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_X_test_lba.csv", 'wb'), protocol=4)
pickle.dump(X_test_size, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_X_test_size.csv", 'wb'), protocol=4)
pickle.dump(y_train_lba, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_y_train_lba.csv", 'wb'), protocol=4)
pickle.dump(y_train_size, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_y_train_size.csv", 'wb'), protocol=4)
pickle.dump(y_test_lba, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_y_test_lba.csv", 'wb'), protocol=4)
pickle.dump(y_test_size, open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/src1_y_test_size.csv", 'wb'), protocol=4)


In [28]:
import pickle 

pickle_off = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_X_train_lba.csv","rb")
X_train_lba = pickle.load(pickle_off)

pickle_off_1 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_X_train_size.csv","rb")
X_train_size = pickle.load(pickle_off_1)

pickle_off_2 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_X_test_lba.csv","rb")
X_test_lba = pickle.load(pickle_off_2)

pickle_off_3 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_X_test_size.csv","rb")
X_test_size = pickle.load(pickle_off_3)

pickle_off_4 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_y_train_lba.csv","rb")
y_train_lba = pickle.load(pickle_off_4)

pickle_off_5 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_y_train_size.csv","rb")
y_train_size = pickle.load(pickle_off_5)

pickle_off_6 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_y_test_lba.csv","rb")
y_test_lba = pickle.load(pickle_off_6)

pickle_off_7 = open("/home/aakashkapoor103/anaconda3/Data/nump_checkpoints/mds0_y_test_size.csv","rb")
y_test_size = pickle.load(pickle_off_7)


In [29]:
X_train_lba.shape, y_train_lba.shape, X_test_lba.shape,  y_test_lba.shape

((908210, 32), (908210, 32, 1), (302693, 32), (302693, 32, 1))

In [30]:
X_train_size.shape, y_train_size.shape, X_test_size.shape,  y_test_size.shape

((908210, 32), (908210, 32, 1), (302693, 32), (302693, 32, 1))

In [31]:
Counter(df['Size_Class'])

Counter({0: 341379,
         1: 11205893,
         2: 3334713,
         3: 5287116,
         4: 578049,
         5: 152495,
         6: 119250,
         7: 248785})

In [32]:
# Two classification outputs
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf






from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate , Dot
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Reshape




# no_docs = len(y_train_lba)
maxlen= 32



# # define two sets of inputs
# inputA = Input(shape=(32,))
# inputB = Input(shape=(32,))
# # inputA = Sequential()
# # inputB = Sequential()
vocabulary_1 = len(Counter(df['ByteOffset_Delta_Class_1001']))
vocabulary_2 = len(Counter(df['Size_Class']))


hidden_size = 1500

# input=Input(shape=(no_docs,maxlen),dtype='float64')
inputA=Input(shape=(maxlen,),dtype='float64')  
inputB=Input(shape=(maxlen,),dtype='float64') 


# the first branch operates on the first input
x = Embedding(input_dim=vocabulary_1,output_dim=hidden_size,input_length=maxlen)(inputA)
x = Model(inputs=inputA, outputs=x)

# # the second branch opreates on the second input
y = Embedding(input_dim=vocabulary_2,output_dim=hidden_size,input_length=maxlen)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.concatenate([x.output, y.output])

lstm1 = LSTM(hidden_size,return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

# create classification output
offset = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_1, activation='softmax'), name='offset')(lstm2)
iosize = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_2, activation='softmax'), name='iosize')(lstm2)




model =Model([inputA,inputB],[offset,iosize]) # combining all into a Keras model

model.compile(optimizer='rmsprop',
              loss={'offset': 'sparse_categorical_crossentropy', 'iosize': 'sparse_categorical_crossentropy'},
              loss_weights={'offset': 2., 'iosize': 1.5},
              metrics={ 'offset': 'categorical_accuracy', 'iosize': 'categorical_accuracy'})
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 1500)     1501500     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 32, 1500)     12000       input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
import time
num_epochs = 50
batch_size = 32

data_path = r'/home/aakashkapoor103/anaconda3/Data/Models/'
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath=data_path + '/src1_1_model-trained.hdf5', verbose=1)


print('Train...')
start_time = time.time()

valid = [X_test_lba,X_test_size],[y_test_lba,y_test_size]

model.fit([X_train_lba,X_train_size],[y_train_lba,y_train_size],
          verbose=1,epochs=75,callbacks=[monitor,checkpointer],batch_size = 512,validation_data = valid)

model.load_weights('best_weights_src1.hdf5') # load weights from best model
print('Done ...!')



In [33]:
model.load_weights('/home/aakashkapoor103/anaconda3/Data/Models/src1_1_model-trained.hdf5') # load weights from best model
print('Done ...!')

OSError: Unable to open file (unable to open file: name = '/home/aakashkapoor103/anaconda3/Data/Models/src1_1_model-trained.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.save('/home/aakashkapoor103/anaconda3/Data/Models/src1_model.h5')

In [ ]:
new_model = keras.models.load_model('/home/aakashkapoor103/anaconda3/Data/Models/src1_model.h5')

In [73]:
import time
start_time = time.time()

pred1,pred2 = new_model.predict([X_test_lba,X_test_size],verbose =1 )
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)



302693/302693 [==============================] - 151s 499us/step
151.01667070388794


In [78]:
elapsed_time/len(X_test_lba)*1000*1000

498.91035043389815

In [ ]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_class'].tolist()
size_test = df[training_pt_1+1:]['Size_class'].tolist()


In [ ]:
len(lba_test)

In [ ]:
len(pred1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


lba_test_final = lba_test[-(len(pred1)):]
lba_size_final = size_test[-(len(pred1)):]

max_lba_accuracy = 0
max_lba_accuracy_pos = 0

max_size_accuracy = 100000
max_size_accuracy_pos = 0

import time
start_time = time.time()

pred_1 = pred1[:,0,:]
pred_2 = pred2[:,0,:]
pred_1 = np.argmax(pred_1, axis=1)
pred_2 = np.argmax(pred_2, axis=1)
elapsed_time = time.time() - start_time
print(elapsed_time)

lba_accuracy = accuracy_score(lba_test_final, pred_1)


size_accuracy = accuracy_score(lba_size_final, pred_2)

print("IO Size Accuracy", str(size_accuracy))
print("Best LBA Delta Accuracy", str(lba_accuracy))

elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
column_names= ['IOSize_pred','IOSize_actual','LBA_pred','LBA_actual']
import pandas as pd
df_res = pd.DataFrame(columns = column_names)

In [ ]:
df_res.head(5)

In [ ]:
df_res.head(5)

In [ ]:
df_res['IOSize_actual'] = lba_size_final

In [ ]:
df_res['IOSize_pred'] = pred_2
df_res['IOSize_actual'] = lba_size_final
df_res['LBA_pred'] = pred_1
df_res['LBA_actual'] = lba_test_final

In [ ]:
df_res['IOSize_log_roundoff'] = df['IOSize_log_roundoff'][-(len(pred1)):].tolist()

In [ ]:
count = 0
lba_hit = 0
block_hit = 0
total_block = 0

while(count<len(df_res)):
    if(df_res['LBA_pred'].iloc[count] == df_res['LBA_actual'].iloc[count]):
        lba_hit = lba_hit +1
        if(df_res['IOSize_actual'].iloc[count] == df_res['IOSize_actual'].iloc[count]):
            block_hit = block_hit + (df_res['IOSize_actual'].iloc[count])
    total_block = total_block + df_res['IOSize_actual'].iloc[count]
    count = count + 1
    
print(block_hit/total_block)
print(lba_hit/count)
        
        
        